In [3]:
#importing packages and modules
import operator
import math
import pandas as pd
import re
import os
import gensim.models as gm
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.text import TextCollection

In [4]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
#nltk.download("RegexpTokenizer")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [75]:
s1= "He said the attackers left behind leaflets urging staff at the Ishtar Sheraton to stop working at the hotel and demanding U.S. forces leave Iraq."
s2 = "He said the attackers left behind leaflets urging workers at the Ishtar Sheraton to stop working at the hotel."
s = [s1,s2]


In [5]:
def preprocessing(s1,s2):
    s = [s1,s2]
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    s1_tokens = tokenizer.tokenize(s1)
    s2_tokens = tokenizer.tokenize(s2)
    s1_tokens_pos = nltk.pos_tag(s1_tokens)
    s2_tokens_pos = nltk.pos_tag(s2_tokens)
    #print (s1_tokens)
    #print (s2_tokens)
    filtered_s1_tokens = [w for w in s1_tokens if not w in stop_words]
    filtered_s2_tokens = [w for w in s2_tokens if not w in stop_words]
    sentences = [filtered_s1_tokens,filtered_s2_tokens]
    word_features = filtered_s1_tokens + filtered_s2_tokens
    return s1_tokens,s2_tokens,filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,sentences
    

In [6]:
def w2vmodel(sentences):
    model = gm.Word2Vec(sentences,min_count = 1)
    return model

In [100]:
s1_tokens,s2_tokens,filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,sentences = preprocessing(s1,s2)
model = w2vmodel(sentences)
final_score = finalsimscore(filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,model)
print (final_score)

0.41989054867307574


C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:20: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


In [7]:
def ri(si,wf,w2v):    
    r = []
    s_sent = si
    word_features = wf
    model = w2v
    for i in word_features:
        j_list=[]
        for j in s_sent:
            j_list.append(model.similarity(i,j))
            #print (j_list)
        max_val = max(j_list)
        #print (max_val)
        pos = j_list.index(max_val)
        r.append(model[i]*pos)
    return (r)


In [8]:


#r1 ri
def wosim(s1_tokens,s2_tokens,wf,wv):
    s1_sent = s1_tokens
    s2_sent = s2_tokens
    word_features = wf
    model = wv
    r1 = ri(s1_sent,word_features,model)
    r2 =ri(s2_sent,word_features,model)
    r1 = np.array(r1)
    r2 = np.array(r2)
    #list(set(r1) - set(r2))
    np_diff =np.sum(np.sum(np.square(np.subtract(r1,r2)),axis=1),axis = 0)
    np_sum = np.sum(np.sum(np.square(np.add(r1,r2)),axis=1),axis=0)
    wo_sim = 1-(np_diff/np_sum)
    return (wo_sim)

    
            

In [9]:
def idf(s,w):
    if w in s[0] or s[1]:
        indices = [z for z, x in enumerate(s) if x == w]
        if w in s[0] and s[1]:
            denom = 2
        else:
            denom = 1
        idf_val = math.log(len(s)/denom,10)
        
        if (idf_val == 0):
            return (0.5)
        else:
            return (idf_val)
    else:
        return (0)
        
        
    

In [10]:
def semsim(a_tokens,b_tokens,wv,a_tokens_pos,b_tokens_pos,s):
    a_tokens_words, a_tokens_tags = zip(*a_tokens_pos)
    b_tokens_words, b_tokens_tags = zip(*b_tokens_pos)
    numerator_list = []
    denominator_list = []
    for i in a_tokens:
        #print ("i")
        #print (i)
        i_tag = a_tokens_tags[a_tokens.index(i)]
        if i_tag in b_tokens_tags:
            indices = [z for z, x in enumerate(b_tokens_tags) if x == i_tag]
            #print ('indices')
            #print (indices)
            sim_list = []
            for k in indices:
                #print ("b")
                #print (b_tokens_words[k])
                if i and b_tokens_words[k] in wv.wv.vocab:
                    #print("sim")
                    sim_list.append(wv.similarity(i,b_tokens_words[k]))
                    #print(sim_list)
                
                #print(sim_list)
                          
        else:
            continue
        #print (sim_list)
        if not sim_list:
            max_sim = 0
        else:
            max_sim = max(sim_list)
            
        numerator_list.append(float(max_sim) * float(idf(s,i)))
        denominator_list.append(idf(s,i))
    semsim_val = sum(numerator_list)/sum(denominator_list)
    return semsim_val
    

In [18]:
#final similarity score
def finalsimscore(s1_tokens,s2_tokens,filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,model):
    sentences = [s1_tokens,s2_tokens]
    s = sentences
    wo_sim = wosim(filtered_s1_tokens,filtered_s2_tokens,word_features,model)
    sem_sim =semsim(filtered_s1_tokens,filtered_s2_tokens,model,s1_tokens_pos,s2_tokens_pos,s) + semsim(filtered_s2_tokens,filtered_s1_tokens,model,s2_tokens_pos,s1_tokens_pos,s)
    alpha = 0.8
    final_score = (alpha * sem_sim) + (1-alpha)*(wo_sim)
    return final_score

In [12]:
def Inputs():

    m1 = "Greetings from Indian Oil we read an article on a certain product of your company. We would like to know the inquire the details of that certain product which you have mentioned on Times Of India. Thank You for cooperating with us"
    m2 = "Thank you for showing interest in the product. We would send you the details of the quotation shortly"
    m3 = "I am attaching a quotation in response and in compliance with the tender mentioned earlier. Please do review this and let me know"
    m4 = "Kindly send us your bank details so that we may make the payment. On receipt of the consignment we will confirm the details of the transaction. The finance is provided by our bankers on credit. The quality should be of the best in the market so that we can expand our business"
    I = [m1,m2,m3,m4]
    return I

In [207]:
#Neural Network
    #take 10 mails from m1 to m10
def 1Inputs():
    
    m1 = "A long time ago, there were four families who lived in a small village in Somalia. The first family would argue all of the time, the second family were very greedy, the third family were always away from the village exploring because they were never happy with what they had or where they lived. But the fourth family were calm and patient, and they enjoyed living in their small community"
    m2 = "One night, the daughter of the third family was out exploring when she discovered a well hidden among some trees in the wilderness. The daughter ran back to her family and told them about the well and so they started to use the well to get their water"
    m3 = "It was not long before the other families heard news of the well, and very soon all four families were using the well to get their water until it was in danger of running dry.This went on for some time, and it was obvious that the water in the well was getting lower and lower, yet none of the families wanted to stop using the well as it was close to the village and meant that they did not have to walk so far to get the water which they used to drink and cook and clean with.One day, the wise chief, who had always known about the secret well, spoke to each family in turn. The chief said to them, Tonight you must stay in your homes. You must not use the well for one whole night, that way the water will have time to rise once more"
    m4 = "Each of the families agreed to stay away from the well, especially as the wise chief warned that there would be a severe punishment for any family who disobeyed this simple rule.But when night fell, the son of the first family could not resist visiting the well as he wanted to make sure he had plenty of water for the following day so that his family would not argue over who would walk the long distance to the usual well used by the rest of the villagers"
    m5 = "He crept out to the well carrying two large buckets and filled them both to the top before returning to his home and hiding the buckets where they would not be seen.Not long after, the son of the second family also crept out to the well and filled two large buckets all the way to the top as he was very greedy and wanted the water for his family alone"
    m6 = "Then the daughter of the third family also crept out to the well as she could not resist exploring at night and reasoned that it was she who had discovered the well in the first place so it was her family who deserved the extra water despite the warning from the wise chief"
    m7 = "The next day, the chief visited the well and was distressed to find that it was completely dry. He waited until he knew that all of the families were away from their homes, then he visited each home in turn"
    m8 = "In the first home he discovered the two buckets, one of which was already empty, but the other still contained the water which was stolen from the well. When he visited the second and third homes he also discovered the buckets of water hidden where nobody would see them. But when he visited the fourth home he discovered that the buckets were dry and realised that the patient family had remained in their beds all night. They had listened to his warning and had stayed away from the well so that the water might rise once more"
    m9 = "The wise chief called all four families to the meeting place in the village where he confronted them about the well. You three families all stole water from the well even though I told you not to, said the chief in a stern voice. I know this because I visited your homes this morning and discovered the buckets of water. Because you defied my instructions you will be forced to remain in your homes for thirty days and nights without food or water as punishment. I hope that you will spend this time thinking about the wrong you have done"
    m10 = "To the fourth family he said, You listened to my simple instructions and stayed in your home last night and did not visit the well. Take this letter and open it when you return to your home"
    m11 = "The fourth family took the letter and returned home. When they opened the letter there was a map inside. The family followed the directions on the map and after travelling for many miles they discovered a well surrounded by an abundance of fruit trees and vegetable plants. There was enough food and water to last the family a whole lifetime.The families who were forced to stay in their homes without food or water learned a valuable lesson that day. They learned that it was always best to listen to the advice of one’s elders and not to take things when you were told not to. They also realised that the fourth family were rewarded for their patience and their willingness to follow the simple rules which benefit a community"
    I = [m1,m2,m3,m4,m5,m6,m7,m8,m9,m10,m11]
    return I


SyntaxError: invalid syntax (<ipython-input-207-2592f69a4eae>, line 3)

In [208]:
#Neural Network
    #take 10 mails from m1 to m10
def 2Inputs():
    m1 = "The theme of this research is mobile transaction processing systems focusing on versatile data sharing mechanisms in volatile mobile environments"
    m2 = "The rapid growth of wireless network technologies and portable computing devices has promoted a new mobile working environment"
    m3 = "A mobile environment is different from the traditional distributed environment due to its unique characteristics the mobility of users or computers the frequent and unpredictable disconnections of wireless networks and the resource constraints of mobile computing devices"
    m4 = "On the one hand the mobile environment promotes a new working model people can carry out their work while being on the move.The environment for accessing and processing information is changing rapidly from stationary and location dependent to mobile and location independent.On the other hand these unique characteristics of the mobile environment pose many challenges to mobile transaction processing systems especially in terms of long delaying periods data unavailability and data inconsistency"
    
    I = [m1,m2,m3,m4]
    
    return I


SyntaxError: invalid syntax (<ipython-input-208-1024c8ad4ede>, line 3)

In [13]:
def iniitialweights():
    I = Inputs()
    W = []
    for  i in I:
        W.append(int(1))
    return W


In [19]:
def hiddenlayer(iteration):
    index_input = 0
    index_end = 0
    I = Inputs()
    if (iteration == 1):
        W = iniitialweights()
    current_mail_val = I[-1]
    current_mail = current_mail_val.split('.')
    sentence_final_score = {}
    for i in range(1,4):  #nodes loop 3
        I = Inputs()
        L = len(I)
        if (L/3 == 0):
            n = L/3
            #print (n)
        else:
            if (i == 1):
                n = math.floor((L/3) + 1)
                index_input = 0
            else:
                n = math.floor(L/3)
        input_node = []
        for index in range(index_input,index_input+n):   #mails loop from I
            input_node.append(I[index])
        #print (current_mail)
        dict_index = -1
        for sentence_current in current_mail:       #sentence current
            #print (sentence_current)
            dict_index = dict_index + 1
            mail_final_score = {}                   
            for mail_val in input_node:               #mail from mails       
                mail = mail_val.split('.')
                sum_final_score = 0
                for sentence_prev in mail:      #sentence from mail
                    #print (sentence_prev)
                    s1_tokens,s2_tokens,filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,sentences = preprocessing(sentence_current,sentence_prev)
                    model = w2vmodel(sentences)
                    final_score = finalsimscore(s1_tokens,s2_tokens,filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,model)
                    sum_final_score = sum_final_score + final_score
                key = I.index(mail_val)
                mail_final_score[key] = sum_final_score
            #print (i)
            #print (sentence_final_score)
            if (i== 1):
                sentence_final_score[current_mail.index(sentence_current)] = mail_final_score
                #print ("i is 1",current_mail_val.index(sentence_current))
            else:
                #print (current_mail_val.index(sentence_current))
                #sentence_final_score[current_mail_val.index(sentence_current)].update(mail_final_score)
                sentence_final_score[dict_index].update(mail_final_score)
        index_input = index_input + n
    #print (sentence_final_score)
    return (sentence_final_score)
    


In [20]:
def final_ouptut(beta,iteration):
    sentence_final_score = hiddenlayer(iteration)
    L= len(Inputs()[-1].split("."))
    retrieval = math.floor(beta * L)
    sum_score = ({k:sum(v.values()) for k,v in sentence_final_score.items()})
    newA = dict(sorted(sum_score.items(), key=operator.itemgetter(1), reverse=True)[:retrieval])
    newA_key = sorted(newA)
    cm = Inputs()[-1].split('.')
    #return {(cm[k])  for k in newA_key}
    return (newA_key)


In [21]:
def summarize():
    
    beta = 0.5
    iteration = 1
    print ("Original Sentence: ")
    print (Inputs()[-1])
    cm = Inputs()[-1].split('.')
    print (" ")

    print ("current sentence : ")
    newA_key = final_ouptut(beta,iteration)
    #print (newA_key)
    for j in newA_key:
        print (cm[j])



In [22]:
summarize()

Original Sentence: 
Kindly send us your bank details so that we may make the payment. On receipt of the consignment we will confirm the details of the transaction. The finance is provided by our bankers on credit. The quality should be of the best in the market so that we can expand our business
 
current sentence : 


C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:20: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


Kindly send us your bank details so that we may make the payment
 The quality should be of the best in the market so that we can expand our business


In [ ]:
dict1 = {'1.2': {1:2,2:3,3:5}, '2.2': {1:2,2:3}, '3.3': {1:2,2:3}}

for k,v in dict1.items():
    for key,val in v.items():
        if key in newdict:
            newdict
        
#print({key:sum(value_1.values()) for k,v in dict1.items() for key,value_1 in v.items()})




In [116]:
sample_dict = {1:{1:6,2:7,3:8},2:{1:4,2:5,3:7}}
sample_dict[1].update({4:3,5:4,6:5})
sample_dict

{1: {1: 6, 2: 7, 3: 8, 4: 3, 5: 4, 6: 5}, 2: {1: 4, 2: 5, 3: 7}}

In [39]:
for i in range(1,4+1):
    print (i)

1
2
3
4


In [113]:
def hiddenlayer_1(iteration):
    for iter_value in range(1,iteration+1):
        index_input = 0
        index_end = 0
        I = Inputs()
        if (iter_value == 1):
            W = iniitialweights()
        current_mail_val = I[-1]
        current_mail = current_mail_val.split('.')
        sentence_final_score = {}
        for i in range(1,4):  #nodes loop 3
            I = Inputs()
            L = len(I)
            if (L/3 == 0):
                n = L/3
                print (n)
            else:
                if (i == 1):
                    n = math.floor((L/3) + 1)
                    index_input = 0
                else:
                    n = math.floor(L/3)
            input_node = []
            for index in range(index_input,index_input+n):   #mails loop from I
                input_node.append(I[index])

            for sentence_current in current_mail:       #sentence current
                mail_final_score = {}                   
                for mail_val in input_node:               #mail from mails       
                    mail = mail_val.split('.')
                    sum_final_score = 0
                    for sentence_prev in mail:      #sentence from mail
                        s1_tokens,s2_tokens,filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,sentences = preprocessing(sentence_current,sentence_prev)
                        model = w2vmodel(sentences)
                        final_score = finalsimscore(filtered_s1_tokens,filtered_s2_tokens,s1_tokens_pos,s2_tokens_pos,word_features,model)
                        sum_final_score = sum_final_score + final_score
                    key = I.index(mail_val)
                    mail_final_score[key] = sum_final_score
                if (i== 1):
                    sentence_final_score[current_mail.index(sentence_current)] = mail_final_score
                else:
                    sentence_final_score[current_mail.index(sentence_current)].update(mail_final_score)
            index_input = index_input + n
            W = W
    return (sentence_final_score)
        
    

    

node
1


C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\user\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:20: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).


node
2
node
3


{0: {0: 1.2044923746847509,
  1: 0.88917064523213396,
  2: 1.4470215424432271,
  3: 0.68228528333286964,
  4: 0.53333814935290924,
  5: 0.29266244818861498,
  6: 0.65681182993322129,
  7: 0.94830013728707785,
  8: 1.4800270771538926,
  9: 0.60366587018442952},
 1: {0: 0.2282598972152014,
  1: 0.32574491765737951,
  2: 0.76945756833882095,
  3: 0.23631498655356192,
  4: 0.18855901095005784,
  5: 0.15343871139978355,
  6: 0.32716721410463256,
  7: 0.84467253326930181,
  8: 0.67357495405882695,
  9: 0.35578288214543846},
 2: {0: 0.98119845582171483,
  1: 1.0280168447147535,
  2: 1.3247769630700659,
  3: 0.58092757347124158,
  4: 0.5033467415828824,
  5: 0.36687880975368492,
  6: 0.54613574295022027,
  7: 0.97263247531454022,
  8: 1.3088753402226605,
  9: 0.35408740142835554},
 3: {0: 1.1649121197271231,
  1: 0.89816903181489161,
  2: 1.6342383068384532,
  3: 0.81538093497453989,
  4: 0.73980709090860275,
  5: 0.40995542709770438,
  6: 0.43418704335408248,
  7: 1.4979726139450842,
  8: 1.4

In [41]:
x = [0,4,34,21,10,6,45,424,90]
array = []
for element in x:
    normalized = (element-min(x))/(max(x)-min(x))
    array.append(normalized)
print (array)


[0.0, 0.009433962264150943, 0.08018867924528301, 0.049528301886792456, 0.02358490566037736, 0.014150943396226415, 0.10613207547169812, 1.0, 0.21226415094339623]


model = w2vmodel(preprocessed_output[sentences])
                    final_score = final_score(preprocessed_output[filtered_s1_tokens],preprocessed_output[filtered_s2_tokens],preprocessed_output[s1_tokens_pos],preprocessed_output[s2_tokens_pos],preprocessed_output[word_features],model)
                    sum_final_score = sum_final_score + final_score
                mail_final_score[key] = sum_final_score
                key = key + 1
            sentence_final_score.append(mail_final_score)
                       
        index_input = index_input + n
        return sentence_final_score
        
        

In [60]:
for i in range(1,4):
    print (i)

1
2
3


In [ ]:
for mail in input_node:
            mail = mail.split('.')
            for sentence_current in current_mail:
                current_mail_sentence_sentence_prev= {}
                for sentence_prev in mail:
                    preprocessed_output = preprocessing(sentence_current,sentence_prev)
                    model = w2vmodel(preprocessed_output[sentences])
                    final_score = final_score(preprocessed_output[filtered_s1_tokens],preprocessed_output[filtered_s2_tokens],preprocessed_output[s1_tokens_pos],preprocessed_output[s2_tokens_pos],preprocessed_output[word_features],model)
                    current_mail_sentence_sentence_prev[sentence_prev.index()] = final_score
                sum_sim_current_mail_sentence_sentence_prev = sum(current_mail_sentence_sentence_prev)
                current_mail_sentence_sim.append(sum_sim_current_mail_sentence_sentence_prev)